In [1]:
colab = True

if colab:
  import torch
  import os
  print("PyTorch has version {}".format(torch.__version__))

  # Install torch geometric
  if 'IS_GRADESCOPE_ENV' not in os.environ:
    !pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.13.1+cu116.html
    !pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.13.1+cu116.html
    !pip install torch-geometric
    !pip install ogb

  !git clone https://github.com/thibautvalour/Graph-Diffusion-Convolution.git
  %cd Graph-Diffusion-Convolution


import os 
import math
import torch
from torch.nn.functional import nll_loss
import copy

from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
import torch_geometric.transforms as T

from models import GCN_Classifier
from utils import train, test
from matrix_format import compute_Tsym, gdc_pagerank

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

PyTorch has version 1.13.1+cu116
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.13.1+cu116.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 30.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.13.1+cu116.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 66.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.0/565.0 KB 12.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 KB 29.4 MB/s eta 0:00:00
  Created wheel for torch-geometric: filename=torch_geometric-2.2.0-py3-none-any.whl size=773302 sha256=4a94d69fc38649c031adf3db690662fc772d73f512b4dc901e623

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 KB 882.0 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7047 sha256=60baa0bbc8795380cfe99c560cf59658ea153df2223072b4b6e7d03c00c3bd4b
  Stored in directory: /root/.cache/pip/wheels/04/bb/0d/2d02ec45f29c48d6192476bfb59c5a0e64b605e7212374dd15
Successfully built littleutils
Cloning into 'Graph-Diffusion-Convolution'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 36 (delta 18), reused 19 (delta 8), pack-reused 0
Unpacking objects: 100% (36/36), 16.34 KiB | 1.49 MiB/s, done.
/content/Graph-Diffusion-Convolution


## Load Data

In [2]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
  dataset_name = 'ogbn-arxiv'
  dataset = PygNodePropPredDataset(name=dataset_name,
                                  transform=T.ToSparseTensor())
  data = dataset[0]

  # Make the adjacency matrix to symmetric
  data.adj_t = data.adj_t.to_symmetric()
  row, col, value = data.adj_t.coo()
  value = torch.ones_like(row,  dtype=torch.float)

  # Create a sparse tensor from the COO format
  indices = torch.stack([row, col])
  A = torch.sparse_coo_tensor(indices, value, 
                              size=[data.num_nodes, data.num_nodes]).to(device)

  # If you use GPU, the device should be cuda
  print('Device: {}'.format(device))

  data = data.to(device)
  split_idx = dataset.get_idx_split()
  train_idx = split_idx['train'].to(device)
  A = A.to(device)

Downloaded 0.08 GB: 100%|██████████| 81/81 [00:09<00:00,  8.85it/s]


Extracting dataset/arxiv.zip


Processing...


Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:00<00:00, 2534.32it/s]


Converting graphs into PyG objects...


100%|██████████| 1/1 [00:00<00:00, 139.98it/s]

Saving...



Done!


Device: cuda


# Define model

In [3]:
Z = A

## GCN

In [4]:
args = {
    'device': device,
    'num_layers': 5,
    'hidden_dim': 264,
    'dropout': 0.5,
    'lr': 0.001,
    'epochs': 100,
    'trans_matrix': A
}

In [5]:
model = GCN_Classifier(input_dim=dataset.num_features,
                       hidden_dim=args['hidden_dim'],
                       output_dim=dataset.num_classes,
                       dropout=args['dropout']).to(args['device'])

optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
evaluator = Evaluator(name='ogbn-arxiv')
loss_fn = nll_loss

best_model = None
best_valid_acc = 0

for epoch in range(1, 1 + args["epochs"]):
  loss = train(model, data, train_idx, args['trans_matrix'], optimizer, loss_fn)
  result = test(model, data, split_idx, args['trans_matrix'], evaluator)
  train_acc, valid_acc, test_acc = result
  if valid_acc > best_valid_acc:
      best_valid_acc = valid_acc
      best_model = copy.deepcopy(model)
  print(f'Epoch: {epoch:02d}, '
        f'Loss: {loss:.4f}, '
        f'Train: {100 * train_acc:.2f}%, '
        f'Valid: {100 * valid_acc:.2f}% '
        f'Test: {100 * test_acc:.2f}%')

/content/Graph-Diffusion-Convolution/models.py:45: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return self.softmax(z3)


Epoch: 01, Loss: 12.0960, Train: 13.01%, Valid: 26.32% Test: 25.57%
Epoch: 02, Loss: 8.3025, Train: 13.33%, Valid: 23.96% Test: 22.86%
Epoch: 03, Loss: 6.7655, Train: 15.11%, Valid: 30.90% Test: 33.33%
Epoch: 04, Loss: 5.7117, Train: 10.63%, Valid: 16.19% Test: 22.99%
Epoch: 05, Loss: 7.3076, Train: 21.63%, Valid: 25.40% Test: 24.55%
Epoch: 06, Loss: 5.2429, Train: 27.78%, Valid: 35.09% Test: 33.23%
Epoch: 07, Loss: 5.2453, Train: 27.80%, Valid: 32.54% Test: 30.18%
Epoch: 08, Loss: 5.7913, Train: 28.69%, Valid: 33.75% Test: 31.26%
Epoch: 09, Loss: 5.7116, Train: 29.32%, Valid: 35.85% Test: 33.57%
Epoch: 10, Loss: 5.0368, Train: 29.37%, Valid: 36.45% Test: 34.00%
Epoch: 11, Loss: 5.0247, Train: 27.63%, Valid: 31.46% Test: 28.64%
Epoch: 12, Loss: 5.0929, Train: 23.22%, Valid: 19.99% Test: 17.67%
Epoch: 13, Loss: 5.3382, Train: 22.06%, Valid: 17.93% Test: 16.16%
Epoch: 14, Loss: 5.5351, Train: 26.09%, Valid: 30.03% Test: 34.38%
Epoch: 15, Loss: 4.7173, Train: 26.57%, Valid: 32.05% Test: 3

## GDN CLASSIQUE

In [6]:
T_sym = compute_Tsym(A)

In [7]:
args = {
    'device': device,
    'num_layers': 5,
    'hidden_dim': 64,
    'dropout': 0.5,
    'lr': 0.001,
    'epochs': 100,
    'trans_matrix': T_sym
}

In [8]:
model = GCN_Classifier(input_dim=dataset.num_features,
                       hidden_dim=args['hidden_dim'],
                       output_dim=dataset.num_classes,
                       dropout=args['dropout']).to(args['device'])

optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
evaluator = Evaluator(name='ogbn-arxiv')
loss_fn = nll_loss

best_model = None
best_valid_acc = 0

for epoch in range(1, 1 + args["epochs"]):
  loss = train(model, data, train_idx, args['trans_matrix'], optimizer, loss_fn)
  result = test(model, data, split_idx, args['trans_matrix'], evaluator)
  train_acc, valid_acc, test_acc = result
  if valid_acc > best_valid_acc:
      best_valid_acc = valid_acc
      best_model = copy.deepcopy(model)
  print(f'Epoch: {epoch:02d}, '
        f'Loss: {loss:.4f}, '
        f'Train: {100 * train_acc:.2f}%, '
        f'Valid: {100 * valid_acc:.2f}% '
        f'Test: {100 * test_acc:.2f}%')

Epoch: 01, Loss: 3.7316, Train: 0.96%, Valid: 1.00% Test: 1.28%
Epoch: 02, Loss: 3.6392, Train: 1.42%, Valid: 1.19% Test: 1.31%
Epoch: 03, Loss: 3.5692, Train: 2.48%, Valid: 2.19% Test: 1.49%
Epoch: 04, Loss: 3.4977, Train: 3.80%, Valid: 4.32% Test: 2.55%
Epoch: 05, Loss: 3.4320, Train: 5.14%, Valid: 7.38% Test: 6.02%
Epoch: 06, Loss: 3.3721, Train: 6.78%, Valid: 10.97% Test: 10.49%
Epoch: 07, Loss: 3.3145, Train: 9.13%, Valid: 14.17% Test: 14.08%
Epoch: 08, Loss: 3.2548, Train: 11.29%, Valid: 15.57% Test: 14.90%
Epoch: 09, Loss: 3.2054, Train: 12.75%, Valid: 16.22% Test: 15.12%
Epoch: 10, Loss: 3.1543, Train: 13.90%, Valid: 16.73% Test: 15.52%
Epoch: 11, Loss: 3.1127, Train: 14.96%, Valid: 17.21% Test: 15.78%
Epoch: 12, Loss: 3.0693, Train: 16.20%, Valid: 17.94% Test: 16.28%
Epoch: 13, Loss: 3.0247, Train: 17.60%, Valid: 18.70% Test: 16.98%
Epoch: 14, Loss: 2.9853, Train: 19.04%, Valid: 19.64% Test: 17.85%
Epoch: 15, Loss: 2.9503, Train: 20.67%, Valid: 21.13% Test: 19.37%
Epoch: 16, L

# Diffusion pagerank (ppr)

In [84]:
import torch
import matplotlib.pyplot as plt,scipy as sp
import numpy as np
from torch.linalg import inv
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def compute_Tsym(adj): 
    ''' adj is a sparse matrix'''
    N = adj.shape[0]
    D = torch.sparse.sum(adj, dim=1).to_dense() # get degree matrix D
    D_sqrt_inv = torch.pow(D, -0.5)
    indices = torch.arange(N).unsqueeze(0).repeat(2, 1).to(device)
    D_sqrt_inv = torch.sparse_coo_tensor(indices, D_sqrt_inv,
                                         size=(N, N))
    
    Tsym = D_sqrt_inv.matmul(adj).matmul(D_sqrt_inv)
    return Tsym

def gdc_pagerank2(A, alpha, eps):
    
    N = A.shape[0]

    # Self-loops
    indices = torch.arange(N).unsqueeze(0).repeat(2, 1).to(device) 
    values = torch.ones(N, dtype=torch.float).to(device) 
    sparse_identity = torch.sparse_coo_tensor(indices, values,
                                             size=(N, N))    
    
    A_loop = A + sparse_identity
    
    # Symmetric transition matrix
    D_loop = torch.sparse.sum(A_loop, dim=1).to_dense()
    D_sqrt_inv = torch.pow(D_loop, -0.5)
    D_sqrt_inv = torch.sparse_coo_tensor(indices, D_sqrt_inv,
                                         size=(N, N))


    T_sym = D_sqrt_inv @ A_loop @ D_sqrt_inv

    # PPR-based diffusion
    test = (sparse_identity-(1-alpha)*T_sym).to_dense()
    print(type(test))
    #test2 = inv(test)
    
    S = alpha * torch.pow(sparse_identity-(1-alpha)*T_sym,-1)

    # TODO : check why negative values are present in S
    # Sparsify using threshold epsilon
    indices = S.indices()
    thresholded_val = S.values() * (S.values() >= eps)
    S_tilde = torch.sparse_coo_tensor(indices, thresholded_val,
                                      size=(N, N))

    # Column-normalized transition matrix on graph S_tilde
    D_tilde_vec = torch.sparse.sum(S_tilde, dim=1).to_dense()
    indices = torch.arange(N).unsqueeze(0).repeat(2, 1).to(device) 
    D_tilde_vec = torch.sparse_coo_tensor(indices, D_tilde_vec,
                                            size=(N, N))
    T_S = S_tilde @ torch.pow(D_tilde_vec, -1)
    
    print("yes")
    
    return T_S



In [85]:
#from matrix_format import compute_Tsym, gdc_pagerank
S_pr = gdc_pagerank2(A, 0.05, 1e-4)

OutOfMemoryError: ignored

In [11]:
args = {
    'device': device,
    'num_layers': 5,
    'hidden_dim': 64,
    'dropout': 0.5,
    'lr': 0.001,
    'epochs': 100,
    'trans_matrix': S_pr
}

In [12]:
model = GCN_Classifier(input_dim=dataset.num_features,
                       hidden_dim=args['hidden_dim'],
                       output_dim=dataset.num_classes,
                       dropout=args['dropout']).to(args['device'])

optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
evaluator = Evaluator(name='ogbn-arxiv')
loss_fn = nll_loss

best_model = None
best_valid_acc = 0

for epoch in range(1, 1 + args["epochs"]):
  loss = train(model, data, train_idx, args['trans_matrix'], optimizer, loss_fn)
  result = test(model, data, split_idx, args['trans_matrix'], evaluator)
  train_acc, valid_acc, test_acc = result
  if valid_acc > best_valid_acc:
      best_valid_acc = valid_acc
      best_model = copy.deepcopy(model)
  print(f'Epoch: {epoch:02d}, '
        f'Loss: {loss:.4f}, '
        f'Train: {100 * train_acc:.2f}%, '
        f'Valid: {100 * valid_acc:.2f}% '
        f'Test: {100 * test_acc:.2f}%')

Epoch: 01, Loss: 3.8805, Train: 2.69%, Valid: 1.76% Test: 1.48%
Epoch: 02, Loss: 3.8244, Train: 6.35%, Valid: 10.36% Test: 9.36%
Epoch: 03, Loss: 3.7764, Train: 10.59%, Valid: 20.33% Test: 18.79%
Epoch: 04, Loss: 3.7262, Train: 12.16%, Valid: 23.04% Test: 21.27%
Epoch: 05, Loss: 3.6777, Train: 13.72%, Valid: 24.33% Test: 22.32%
Epoch: 06, Loss: 3.6278, Train: 14.63%, Valid: 24.51% Test: 22.37%
Epoch: 07, Loss: 3.5808, Train: 14.76%, Valid: 23.91% Test: 21.71%
Epoch: 08, Loss: 3.5353, Train: 14.77%, Valid: 22.99% Test: 20.79%
Epoch: 09, Loss: 3.4911, Train: 14.81%, Valid: 22.35% Test: 20.04%
Epoch: 10, Loss: 3.4507, Train: 15.16%, Valid: 22.05% Test: 19.56%
Epoch: 11, Loss: 3.4085, Train: 15.85%, Valid: 21.97% Test: 19.49%
Epoch: 12, Loss: 3.3658, Train: 16.83%, Valid: 22.16% Test: 19.68%
Epoch: 13, Loss: 3.3234, Train: 18.09%, Valid: 22.68% Test: 20.04%
Epoch: 14, Loss: 3.2879, Train: 19.43%, Valid: 23.43% Test: 20.59%
Epoch: 15, Loss: 3.2490, Train: 20.86%, Valid: 24.24% Test: 21.29%


# Diffusion Heat

## from Scratch 

In [58]:
def gdc_heat(A, t, sum_limit, eps):

    N = A.shape[0]
    # Self-loops
    indices = torch.arange(N).unsqueeze(0).repeat(2, 1).to(device) 
    values = torch.ones(N, dtype=torch.float).to(device) 
    sparse_identiy = torch.sparse_coo_tensor(indices, values,
                                             size=(N, N))    
    
    A_loop = A + sparse_identiy
    
    # Symmetric transition matrix
    D_loop = torch.sparse.sum(A_loop, dim=1).to_dense()
    D_sqrt_inv = torch.pow(D_loop, -0.5)
    D_sqrt_inv = torch.sparse_coo_tensor(indices, D_sqrt_inv,
                                         size=(N, N))

    T_sym = D_sqrt_inv @ A_loop @ D_sqrt_inv

    S = torch.sparse_coo_tensor(size=(N, N)).to(device)
    T_k = sparse_identiy
    
    for k in range(sum_limit):
      heat_coeff = math.exp(-t * t**k / math.factorial(k))
      rlt = heat_coeff * T_k
      T_sym= T_sym.coalesce()
      T_sym = torch.sparse_coo_tensor(T_sym.indices(), torch.pow(T_sym.values(),k),  T_sym.shape) 
      S += rlt @ T_sym
    
    #T_k = T_k @ T_sym

    # TODO : check why negative values are present in S
    # Sparsify using threshold epsilon
    indices =  S.coalesce().indices()
    thresholded_val = S.coalesce().values() * (S.coalesce().values() >= eps)
    S_tilde = torch.sparse_coo_tensor(indices, thresholded_val,
                                      size=(N, N))

    # Column-normalized transition matrix on graph S_tilde
    D_tilde_vec = torch.sparse.sum(S_tilde, dim=1).to_dense()
    indices = torch.arange(N).unsqueeze(0).repeat(2, 1).to(device) 
    D_tilde_vec = torch.sparse_coo_tensor(indices, D_tilde_vec,
                                            size=(N, N))
    T_S = S_tilde @ torch.pow(D_tilde_vec, -1)
    
    return T_S


# With pytorch function

In [51]:
from torch_geometric.utils import (
    add_self_loops,
    coalesce,
    is_undirected,
    scatter,
    to_dense_adj,)

def expm(matrix) :
  mat = matrix.coalesce()
  diff_mat = torch.sparse_coo_tensor(mat.indices(), mat.values().exp(), mat.shape)      
  #diff_mat_np = expm(matrix.cpu().numpy())
  #diff_mat = torch.Tensor(diff_mat_np).to(matrix.device)
  return diff_mat


def gdc_heat2(A, t,k, sum_limit, eps):
  num_nodes = A.shape[0]
  indices = torch.arange(num_nodes).unsqueeze(0).repeat(2, 1).to(device) 
  values = torch.ones(num_nodes, dtype=torch.float).to(device) 

  edge_index, edge_weight = add_self_loops(indices, values,fill_value=-1, num_nodes=num_nodes)
  edge_weight = t * edge_weight
  mat = torch.sparse_coo_tensor(edge_index, edge_weight,size=(num_nodes, num_nodes))
  print(mat)
  #mat = to_dense_adj(edge_index, edge_attr=edge_weight).squeeze()
  #undirected = is_undirected(edge_index, edge_weight, num_nodes)
     
  #diff_matrix = expm(mat, False)

  return mat




In [59]:
S = gdc_heat(A, 3, 25, 1e-4)

In [60]:
args = {
    'device': device,
    'num_layers': 5,
    'hidden_dim': 64,
    'dropout': 0.5,
    'lr': 0.001,
    'epochs': 150,
    'trans_matrix': S
}

In [61]:
model = GCN_Classifier(input_dim=dataset.num_features,
                       hidden_dim=args['hidden_dim'],
                       output_dim=dataset.num_classes,
                       dropout=args['dropout']).to(args['device'])

optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
evaluator = Evaluator(name='ogbn-arxiv')
loss_fn = nll_loss

best_model = None
best_valid_acc = 0

for epoch in range(1, 1 + args["epochs"]):
  loss = train(model, data, train_idx, args['trans_matrix'], optimizer, loss_fn)
  result = test(model, data, split_idx, args['trans_matrix'], evaluator)
  train_acc, valid_acc, test_acc = result
  if valid_acc > best_valid_acc:
      best_valid_acc = valid_acc
      best_model = copy.deepcopy(model)
  print(f'Epoch: {epoch:02d}, '
        f'Loss: {loss:.4f}, '
        f'Train: {100 * train_acc:.2f}%, '
        f'Valid: {100 * valid_acc:.2f}% '
        f'Test: {100 * test_acc:.2f}%')

Epoch: 01, Loss: 3.7595, Train: 0.41%, Valid: 0.39% Test: 0.36%
Epoch: 02, Loss: 3.6970, Train: 0.36%, Valid: 0.33% Test: 0.28%
Epoch: 03, Loss: 3.6474, Train: 0.30%, Valid: 0.29% Test: 0.24%
Epoch: 04, Loss: 3.6074, Train: 0.29%, Valid: 0.25% Test: 0.21%
Epoch: 05, Loss: 3.5696, Train: 0.28%, Valid: 0.24% Test: 0.20%
Epoch: 06, Loss: 3.5377, Train: 0.31%, Valid: 0.30% Test: 0.25%
Epoch: 07, Loss: 3.5049, Train: 0.62%, Valid: 0.58% Test: 0.44%
Epoch: 08, Loss: 3.4704, Train: 1.46%, Valid: 1.27% Test: 0.93%
Epoch: 09, Loss: 3.4391, Train: 2.84%, Valid: 2.22% Test: 1.73%
Epoch: 10, Loss: 3.4107, Train: 4.13%, Valid: 2.97% Test: 2.34%
Epoch: 11, Loss: 3.3792, Train: 4.76%, Valid: 3.31% Test: 2.57%
Epoch: 12, Loss: 3.3478, Train: 5.06%, Valid: 3.48% Test: 2.70%
Epoch: 13, Loss: 3.3199, Train: 5.63%, Valid: 3.80% Test: 2.83%
Epoch: 14, Loss: 3.2887, Train: 6.37%, Valid: 4.32% Test: 3.02%
Epoch: 15, Loss: 3.2617, Train: 7.25%, Valid: 5.15% Test: 3.37%
Epoch: 16, Loss: 3.2320, Train: 8.29%, V